In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cab

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from sklearn.preprocessing import KBinsDiscretizer

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

数据处理

In [2]:
test_data = pd.read_csv('data/test/000000000000.csv', sep='\t')
test_data.shape 

(160973, 80)

In [3]:
train_data = pd.read_csv('./data/train.csv', sep='\t')
train_data.shape 

(3485852, 82)

In [4]:
train_data = train_data.replace([np.inf, -np.inf], np.nan).fillna(0)
test_data = test_data.replace([np.inf, -np.inf], np.nan).fillna(0)

In [5]:
data = pd.concat([train_data, test_data])

In [6]:
cat_features = ['f_{}'.format(i) for i in range(1, 42)]
bin_features = ['f_{}'.format(i) for i in range(33, 42)]
num_features = ['f_{}'.format(i) for i in range(42, 80)]
date_features = ['f_1']

In [7]:
for f in cat_features:
    le = LabelEncoder()
    data[f] = le.fit_transform(data[f])

In [8]:
dense_bins = 10

In [9]:
add_cat_features = []
for fea in tqdm(num_features, total=len(num_features)):
    discretizer = KBinsDiscretizer(n_bins=dense_bins, encode='ordinal', strategy='kmeans')  # 等频quantile，等宽uniform
    data[fea + '_encode'] = discretizer.fit_transform(np.array(data[fea].tolist()).reshape(-1, 1))
    add_cat_features.append(fea + '_encode')

100%|██████████| 38/38 [01:52<00:00,  2.95s/it]


In [10]:
# zero_count = data.eq(0).sum()  # 计算每列中值为 0 的数量
# zero_ratio = zero_count / len(data)  # 计算每列中值为 0 的占比
# print(zero_ratio)

In [11]:
data.shape 

(3646825, 120)

In [12]:
train_data[train_data['f_0'] == 64505]

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
2596150,64505,58,3346,22294,6767,25604,5040,27941,19203,31372,...,0.571121,0.0,0.0,0.269948,0.0,0.0,0.0,0.0,0,0


In [13]:
test_data[test_data['f_0'] == 64505]

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79
0,64505,67,26325,7152,21563,19475,31440,27941,21621,14659,...,1.519085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
data.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_70_encode,f_71_encode,f_72_encode,f_73_encode,f_74_encode,f_75_encode,f_76_encode,f_77_encode,f_78_encode,f_79_encode
0,2541188,12,114,4,191,4,155,0,1,4,...,2.0,2.0,4.0,4.0,3.0,9.0,7.0,0.0,0.0,0.0
1,2541440,15,21,4,390,3,559,0,2,3,...,1.0,0.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,0.0
2,2541480,20,131,4,228,3,3261,0,1,6,...,3.0,0.0,1.0,0.0,0.0,9.0,7.0,0.0,0.0,0.0
3,2541780,18,73,2,317,3,3843,0,2,0,...,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
4,2541833,15,44,4,519,3,5179,0,4,3,...,2.0,0.0,0.0,0.0,3.0,9.0,7.0,0.0,0.0,0.0


In [15]:
# zero_ratio = zero_ratio.sort_values(ascending=False)
# zero_ratio.apply(lambda x: round(x, 5))
# zero_ratio.to_csv('./output/zero.csv')

In [16]:
# for i in tqdm(range(len(num_features)), total=len(num_features)):
#     for j in range(i + 1, len(num_features)):
#         data[f'{num_features[i]}+{num_features[j]}'] = data[num_features[i]] + data[num_features[j]]
#         data[f'{num_features[i]}-{num_features[j]}'] = data[num_features[i]] - data[num_features[j]]
#         data[f'{num_features[i]}*{num_features[j]}'] = data[num_features[i]] * data[num_features[j]]
#         data[f'{num_features[i]}/{num_features[j]}'] = data[num_features[i]] / (data[num_features[j]] + 1e-7)
# data.shape 

In [17]:
# for i in tqdm(range(len(bin_features)), total=len(bin_features)):
#     for j in range(i + 1, len(bin_features)):
#         data[f'{bin_features[i]}and{bin_features[j]}'] = data[bin_features[i]] & data[bin_features[j]]
#         data[f'{bin_features[i]}or{bin_features[j]}'] = data[bin_features[i]] | data[bin_features[j]]
#         data[f'{bin_features[i]}xor{bin_features[j]}'] = data[bin_features[i]] ^ data[bin_features[j]]
# data.shape 

In [18]:
# np.save('data/data_aug.npy', data)

In [19]:
features = [fe for fe in data.columns if fe not in ['is_clicked', 'is_installed', 'f_0', 'label']]
len(features)

117

In [20]:
train = data[~data['is_clicked'].isna()]
test = data[data['is_clicked'].isna()]
is_clicked_label = train['is_clicked'].astype('int')
is_installed_label = train['is_installed'].astype('int')

train.shape, test.shape, is_clicked_label.shape, is_installed_label.shape 

((3485852, 120), (160973, 120), (3485852,), (3485852,))

In [21]:
seed = 42
K = 5

In [22]:
lgb_params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    # 'metric': 'binary_logloss',  # auc??
    'metric': 'binary_logloss',
    'learning_rate': 0.05,
    'num_leaves': 2 ** 6,
    'max_depth': 8,
    'tree_learner': 'serial',
    'colsample_bytree': 0.8,
    'subsample_freq': 1,
    'subsample': 0.8,
    'num_boost_round': 3000,
    'max_bin': 255,
    'verbose': -1,
    'nthread' : -1,
    'seed': seed,
    'bagging_seed': seed,
    'feature_fraction_seed': seed,
    'early_stopping_rounds': 100,
    # 'device': 'gpu',  # 设置使用 GPU 加速
    # 'gpu_platform_id': 0,  # 设置 GPU 平台 id
    # 'gpu_device_id': 0  # 设置 GPU 设备 id
}

is_clicked

In [21]:
label = is_clicked_label

In [14]:
label = is_clicked_label

KF = StratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
feat_imp_lgb_cli = list()

oof_lgb_cli = np.zeros(len(train))
predictions_lgb_cli = np.zeros((len(test)))
print(len(features))

# 模型训练
for fold_, (trn_idx, val_idx) in enumerate(KF.split(train.values, label.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=label.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=label.iloc[val_idx])
    
    num_round = 3000
    clf = lgb.train(
        lgb_params,
        trn_data,
        num_round,
        valid_sets=[trn_data, val_data],
        verbose_eval=300,
        early_stopping_rounds=100,
    )

    oof_lgb_cli[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions_lgb_cli[:] += clf.predict(test[features], num_iteration=clf.best_iteration) / K
    feat_imp_lgb_cli.append(clf.feature_importance())

79
fold n°0
[LightGBM] [Warning] num_threads is set with nthread=-1, will be overridden by n_jobs=30. Current value: num_threads=30
Training until validation scores don't improve for 100 rounds
[300]	training's binary_logloss: 0.326966	valid_1's binary_logloss: 0.327212
[600]	training's binary_logloss: 0.315277	valid_1's binary_logloss: 0.317277
[900]	training's binary_logloss: 0.310265	valid_1's binary_logloss: 0.314412
[1200]	training's binary_logloss: 0.306852	valid_1's binary_logloss: 0.313149
[1500]	training's binary_logloss: 0.303901	valid_1's binary_logloss: 0.312309
[1800]	training's binary_logloss: 0.301191	valid_1's binary_logloss: 0.31171
[2100]	training's binary_logloss: 0.298683	valid_1's binary_logloss: 0.311242
[2400]	training's binary_logloss: 0.296324	valid_1's binary_logloss: 0.310938
[2700]	training's binary_logloss: 0.294051	valid_1's binary_logloss: 0.310649
[3000]	training's binary_logloss: 0.291819	valid_1's binary_logloss: 0.310385
Did not meet early stopping. B

In [15]:
logloss = metrics.log_loss(label, [1 if i >= 0.5 else 0 for i in oof_lgb_cli], labels=[0, 1])
logloss2 = metrics.log_loss(label, oof_lgb_cli)

acc = metrics.roc_auc_score(label, oof_lgb_cli)
precision = metrics.precision_score(label, [1 if i >= 0.5 else 0 for i in oof_lgb_cli])
recall = metrics.recall_score(label, [1 if i >= 0.5 else 0 for i in oof_lgb_cli])
f1 = metrics.f1_score(label, [1 if i >= 0.5 else 0 for i in oof_lgb_cli])

print(f"Logloss: {logloss:.4f}, {logloss2:.4f}, AUC: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Logloss: 4.0459, 0.3113, AUC: 0.8730, Precision: 0.8972, Recall: 0.5276, F1 Score: 0.6644


In [21]:
avg_imp_cli = pd.DataFrame(feat_imp_lgb_cli, columns = features).apply(np.mean, axis = 0).sort_values(ascending=False)
avg_imp_cli.describe()

count       79.000000
mean      2390.579747
std       2535.470191
min          0.000000
25%        309.900000
50%       1537.000000
75%       3903.800000
max      10211.200000
dtype: float64

In [22]:
avg_imp_cli

f_4     10211.2
f_11     9546.8
f_15     9148.8
f_42     8856.2
f_6      8218.0
         ...   
f_29        0.0
f_7         0.0
f_28        0.0
f_27        0.0
f_26        0.0
Length: 79, dtype: float64

# is_installed

In [23]:
label = is_installed_label

lightgbm

In [24]:
lgb_params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    # 'metric': 'binary_logloss',  # auc??
    'metric': 'binary_logloss',
    'learning_rate': 0.05,
    'num_leaves': 2 ** 6,
    'max_depth': 8,
    'tree_learner': 'serial',
    'colsample_bytree': 0.8,
    'subsample_freq': 1,
    'subsample': 0.8,
    'num_boost_round': 3000,
    'max_bin': 255,
    'verbose': -1,
    'nthread' : -1,
    'seed': seed,
    'bagging_seed': seed,
    'feature_fraction_seed': seed,
    'early_stopping_rounds': 100,
    # 'device': 'gpu',  # 设置使用 GPU 加速
    # 'gpu_platform_id': 0,  # 设置 GPU 平台 id
    # 'gpu_device_id': 0  # 设置 GPU 设备 id
}

In [25]:
features = select_fea_lgb

NameError: name 'select_fea_lgb' is not defined

In [26]:
label = is_installed_label

KF = StratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
feat_imp_lgb_ins = list()

oof_lgb_ins = np.zeros(len(train))
predictions_lgb_ins = np.zeros((len(test)))
print(len(features))

# 模型训练
for fold_, (trn_idx, val_idx) in enumerate(KF.split(train.values, label.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=label.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=label.iloc[val_idx])
    
    num_round = 3000
    clf = lgb.train(
        lgb_params,
        trn_data,
        num_round,
        valid_sets=[trn_data, val_data],
        verbose_eval=300,
        early_stopping_rounds=100,
    )

    oof_lgb_ins[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions_lgb_ins[:] += clf.predict(test[features], num_iteration=clf.best_iteration) / K
    feat_imp_lgb_ins.append(clf.feature_importance())

117
fold n°0
Training until validation scores don't improve for 100 rounds
[300]	training's binary_logloss: 0.299157	valid_1's binary_logloss: 0.300993
[600]	training's binary_logloss: 0.291614	valid_1's binary_logloss: 0.295626
[900]	training's binary_logloss: 0.287519	valid_1's binary_logloss: 0.293822
[1200]	training's binary_logloss: 0.283973	valid_1's binary_logloss: 0.292536
[1500]	training's binary_logloss: 0.281049	valid_1's binary_logloss: 0.291817
[1800]	training's binary_logloss: 0.278411	valid_1's binary_logloss: 0.29134
[2100]	training's binary_logloss: 0.275853	valid_1's binary_logloss: 0.290892
[2400]	training's binary_logloss: 0.273516	valid_1's binary_logloss: 0.290671
[2700]	training's binary_logloss: 0.271277	valid_1's binary_logloss: 0.290457
[3000]	training's binary_logloss: 0.269068	valid_1's binary_logloss: 0.290264
Did not meet early stopping. Best iteration is:
[3000]	training's binary_logloss: 0.269068	valid_1's binary_logloss: 0.290264
fold n°1
Training until

In [27]:
# logloss = metrics.log_loss(label, [1 if i >= 0.5 else 0 for i in oof_lgb_ins], labels=[0, 1])
logloss = metrics.log_loss(label, oof_lgb_ins)
acc = metrics.roc_auc_score(label, oof_lgb_ins)
precision = metrics.precision_score(label, [1 if i >= 0.5 else 0 for i in oof_lgb_ins])
recall = metrics.recall_score(label, [1 if i >= 0.5 else 0 for i in oof_lgb_ins])
f1 = metrics.f1_score(label, [1 if i >= 0.5 else 0 for i in oof_lgb_ins])

print(f"Logloss: {logloss:.4f}, AUC: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Logloss: 0.2903, AUC: 0.8838, Precision: 0.7904, Recall: 0.3897, F1 Score: 0.5220


In [28]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = np.random.random((test_data.shape[0]))
submission["is_installed"] = predictions_lgb_ins
submission.to_csv('./output/lgb_42_sep_train_logloss_float_{}.csv'.format(len(features)), index=False, sep='\t')

In [29]:
avg_imp_lgb_ins = pd.DataFrame(feat_imp_lgb_ins, columns = features).apply(np.mean, axis = 0).sort_values(ascending=False)
avg_imp_lgb_ins.describe()

count     117.000000
mean     1613.825641
std      2203.685731
min         0.000000
25%       110.600000
50%       356.800000
75%      2694.000000
max      9222.000000
dtype: float64

In [30]:
select_fea_lgb = []
for i in range(len(avg_imp_lgb_ins)):
    if avg_imp_lgb_ins.iloc[i] >= 3875.300000:
        select_fea_lgb.append(avg_imp_lgb_ins.index[i])
print(len(select_fea_lgb))
print(select_fea_lgb)

19
['f_15', 'f_42', 'f_11', 'f_6', 'f_4', 'f_57', 'f_1', 'f_59', 'f_58', 'f_61', 'f_62', 'f_51', 'f_55', 'f_67', 'f_65', 'f_17', 'f_2', 'f_64', 'f_63']


In [ ]:
# TODO: 特征筛选

xgboost

In [28]:
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'random_state': seed,
    'tree_method': 'hist',
    'booster': 'gbtree',
    'n_estimators': 1000, 
    'learning_rate': 0.1,
    'max_depth': 6,
    'min_child_weight': 12, 
    'subsample': 0.8,
    'colsample_bytree': 1.0, 
    'alpha': 0.7,
    'lambda': 0.1, 
    'gamma': 0.1, 
    "nthread": -1
}

In [31]:
xgb_params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'gamma': 1,
    'min_child_weight': 1.5,
    'max_depth': 6,
    'lambda': 10,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'colsample_bylevel': 0.7,
    'eta': 0.05,
    'tree_method': 'exact',
    'seed': seed,
    'nthread': -1
}

In [51]:
features = select_fea_xgb

In [32]:
label = is_installed_label

KF = StratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
feat_imp_xgb_ins = list()

oof_xgb_ins = np.zeros(len(train))
predictions_xgb_ins = np.zeros((len(test)))
print(len(features))

# 模型训练
for fold_, (trn_idx, val_idx) in enumerate(KF.split(train.values, label.values)):
    print("fold n°{}".format(fold_))
    X_train, X_val = train.iloc[trn_idx][features], train.iloc[val_idx][features]
    y_train, y_val = label.iloc[trn_idx], label.iloc[val_idx]
    trn_data = xgb.DMatrix(X_train, label=y_train)
    val_data = xgb.DMatrix(X_val, label=y_val)

    num_round = 3000
    clf = xgb.train(
        xgb_params,
        trn_data,
        num_round,
        evals = [(trn_data, 'train'), (val_data, 'val')],
        verbose_eval=300,
        early_stopping_rounds=100,
    )

    oof_xgb_ins[val_idx] = clf.predict(val_data, iteration_range=(0, clf.best_iteration))
    predictions_xgb_ins[:] += clf.predict(xgb.DMatrix(test[features]), iteration_range=(0, clf.best_iteration)) / K
    feat_imp_xgb_ins.append(clf.get_score())

117
fold n°0
[0]	train-logloss:0.66547	val-logloss:0.66547
[300]	train-logloss:0.30737	val-logloss:0.30856
[600]	train-logloss:0.29827	val-logloss:0.30072
[900]	train-logloss:0.29372	val-logloss:0.29731
[1200]	train-logloss:0.29063	val-logloss:0.29534
[1500]	train-logloss:0.28828	val-logloss:0.29422
[1800]	train-logloss:0.28616	val-logloss:0.29325
[2100]	train-logloss:0.28442	val-logloss:0.29263
[2400]	train-logloss:0.28281	val-logloss:0.29221
[2700]	train-logloss:0.28132	val-logloss:0.29184
[2999]	train-logloss:0.27995	val-logloss:0.29157
fold n°1
[0]	train-logloss:0.66519	val-logloss:0.66519
[300]	train-logloss:0.30746	val-logloss:0.30832
[600]	train-logloss:0.29869	val-logloss:0.30080
[900]	train-logloss:0.29405	val-logloss:0.29737
[1200]	train-logloss:0.29089	val-logloss:0.29537
[1500]	train-logloss:0.28845	val-logloss:0.29408
[1800]	train-logloss:0.28627	val-logloss:0.29306
[2100]	train-logloss:0.28451	val-logloss:0.29248
[2400]	train-logloss:0.28286	val-logloss:0.29195
[2700]	tra

In [33]:
# logloss = metrics.log_loss(label, [1 if i >= 0.5 else 0 for i in oof_xgb_ins], labels=[0, 1])
logloss = metrics.log_loss(label, oof_xgb_ins)
acc = metrics.roc_auc_score(label, oof_xgb_ins)
precision = metrics.precision_score(label, [1 if i >= 0.5 else 0 for i in oof_xgb_ins])
recall = metrics.recall_score(label, [1 if i >= 0.5 else 0 for i in oof_xgb_ins])
f1 = metrics.f1_score(label, [1 if i >= 0.5 else 0 for i in oof_xgb_ins])

print(f"Logloss: {logloss:.4f}, AUC: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Logloss: 0.2917, AUC: 0.8826, Precision: 0.7907, Recall: 0.3830, F1 Score: 0.5161


In [34]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = np.random.random((test_data.shape[0]))
submission["is_installed"] = predictions_xgb_ins
submission.to_csv('./output/xgb_42_sep_train_logloss_float_{}_newparam.csv'.format(len(features)), index=False, sep='\t')

In [35]:
avg_imp_xgb_ins = pd.DataFrame(feat_imp_xgb_ins, columns = features).apply(np.mean, axis = 0).sort_values(ascending=False)
avg_imp_xgb_ins.describe()

count     116.000000
mean     1413.609770
std      1861.148202
min         1.000000
25%       143.250000
50%       476.400000
75%      2351.950000
max      7971.200000
dtype: float64

In [36]:
select_fea_xgb = []
for i in range(len(avg_imp_xgb_ins)):
    if avg_imp_xgb_ins.iloc[i] >= 3514.600000:
        select_fea_xgb.append(avg_imp_xgb_ins.index[i])
print(len(select_fea_xgb))
print(select_fea_xgb)

13
['f_6', 'f_15', 'f_11', 'f_4', 'f_42', 'f_59', 'f_1', 'f_58', 'f_57', 'f_51', 'f_67', 'f_65', 'f_2']


catboost

In [37]:
cbc_params = {
    'random_state': seed,
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'verbose': False,
    'learning_rate': 0.05,
    'depth': 5,
    'rsm': 0.2020238568794654,
    # 'min_data_in_leaf': 255,
    # 'l2_leaf_reg': 5,
    # 'subsample': 0.7,
    # 'use_best_model': True,
    # 'max_leaves': 12,
    # 'metric_period': 500,
    'n_estimators': 3000,
    'thread_count': -1
}

In [57]:
features = select_fea_cbc

In [60]:
cat_features = [f for f in cat_features if f in select_fea_cbc]

In [38]:
KF = StratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
feat_imp_cbc_ins = list()

oof_cbc_ins = np.zeros(len(train))
predictions_cbc_ins = np.zeros((len(test)))
print(len(features))

model = CatBoostClassifier(**cbc_params)

# 模型训练
for fold_, (trn_idx, val_idx) in enumerate(KF.split(train.values, label.values)):
    print("fold n°{}".format(fold_))

    X_train, X_val = train.iloc[trn_idx][features], train.iloc[val_idx][features]
    y_train, y_val = label.iloc[trn_idx], label.iloc[val_idx]
    
    model.fit(X_train, y_train, eval_set=(X_val, y_val), 
              cat_features=cat_features,
              early_stopping_rounds=100, verbose=300, use_best_model=True)

    oof_cbc_ins[val_idx] += (model.predict_proba(X_val)[:, 1])
    predictions_cbc_ins += (model.predict_proba(test[features])[:, 1]) / K
    feat_imp_cbc_ins.append(model.feature_importances_)

117
fold n°0
0:	learn: 0.6534025	test: 0.6534221	best: 0.6534221 (0)	total: 829ms	remaining: 41m 25s
300:	learn: 0.3010529	test: 0.3001665	best: 0.3001665 (300)	total: 4m 51s	remaining: 43m 31s
600:	learn: 0.2938884	test: 0.2930073	best: 0.2930073 (600)	total: 9m 55s	remaining: 39m 37s
900:	learn: 0.2916883	test: 0.2909139	best: 0.2909139 (900)	total: 14m 47s	remaining: 34m 26s
1200:	learn: 0.2903642	test: 0.2897239	best: 0.2897239 (1200)	total: 19m 47s	remaining: 29m 38s
1500:	learn: 0.2895748	test: 0.2890928	best: 0.2890928 (1500)	total: 24m 45s	remaining: 24m 43s
1800:	learn: 0.2888977	test: 0.2885608	best: 0.2885608 (1800)	total: 30m	remaining: 19m 58s
2100:	learn: 0.2883433	test: 0.2881662	best: 0.2881662 (2100)	total: 35m 7s	remaining: 15m 1s
2400:	learn: 0.2878501	test: 0.2878213	best: 0.2878213 (2400)	total: 40m 11s	remaining: 10m 1s
2700:	learn: 0.2874355	test: 0.2875655	best: 0.2875655 (2700)	total: 45m 8s	remaining: 4m 59s
2999:	learn: 0.2870611	test: 0.2873420	best: 0.28734

In [39]:
# logloss = metrics.log_loss(label, [1 if i >= 0.5 else 0 for i in oof_cbc_ins], labels=[0, 1])
logloss = metrics.log_loss(label, oof_cbc_ins)
acc = metrics.roc_auc_score(label, oof_cbc_ins)
precision = metrics.precision_score(label, [1 if i >= 0.5 else 0 for i in oof_cbc_ins])
recall = metrics.recall_score(label, [1 if i >= 0.5 else 0 for i in oof_cbc_ins])
f1 = metrics.f1_score(label, [1 if i >= 0.5 else 0 for i in oof_cbc_ins])

print(f"Logloss: {logloss:.4f}, AUC: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Logloss: 0.2874, AUC: 0.8863, Precision: 0.7833, Recall: 0.4090, F1 Score: 0.5374


In [40]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = np.random.random((test_data.shape[0]))
submission["is_installed"] = predictions_cbc_ins
submission.to_csv('./output/cbc_42_sep_train_logloss_float_{}.csv'.format(len(features)), index=False, sep='\t')

In [41]:
avg_imp_cbc_ins = pd.DataFrame(feat_imp_cbc_ins, columns = features).apply(np.mean, axis = 0).sort_values(ascending=False)
avg_imp_cbc_ins.describe()

count    117.000000
mean       0.854701
std        2.258121
min        0.000000
25%        0.015990
50%        0.192809
75%        0.556579
max       17.879745
dtype: float64

In [42]:
select_fea_cbc = []
for i in range(len(avg_imp_cbc_ins)):
    if avg_imp_cbc_ins.iloc[i] >= 0.976061:
        select_fea_cbc.append(avg_imp_cbc_ins.index[i])
print(len(select_fea_cbc))
print(select_fea_cbc)

15
['f_4', 'f_1', 'f_2', 'f_59_encode', 'f_6', 'f_59', 'f_20', 'f_19', 'f_11', 'f_42', 'f_10', 'f_68', 'f_12', 'f_21', 'f_57']


# 模型融合

In [3]:
file_list = ['NFM_2023_79_4_1', 'cbc_42_sep_train_logloss_float']
weight_list = [1, 1]
res_list = []
for file in file_list:
    res_list.append(pd.read_csv('./output/{}.csv'.format(file), sep='\t')['is_installed'].values)

In [4]:
res_wei = 0
for i, res_ in enumerate(res_list):
    res_wei += res_ * weight_list[i]
res_wei /= sum(weight_list)

In [5]:
res_list

[array([0.34951466, 0.38672057, 0.01923786, ..., 0.03878488, 0.00983924,
        0.03923493]),
 array([0.34246523, 0.49886412, 0.12743884, ..., 0.20235799, 0.24234735,
        0.1216858 ])]

In [6]:
res_wei

array([0.34598995, 0.44279234, 0.07333835, ..., 0.12057144, 0.12609329,
       0.08046036])

In [9]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = np.random.random((test_data.shape[0]))
submission["is_installed"] = res_wei
submission.to_csv('./output/cbc_NFM_1_1.csv', index=False, sep='\t')

# 输出

In [26]:
seed

42

In [33]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = np.random.random((test_data.shape[0]))
submission["is_installed"] = predictions_cbc_ins
submission.to_csv('./output/cbc_42_sep_train_logloss_float.csv', index=False, sep='\t')

In [24]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = predictions_lgb_cli
submission["is_installed"] = predictions_lgb_ins
submission.to_csv('./output/lgb_42_sep_train_logloss_float.csv', index=False, sep='\t')

In [26]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = np.round(predictions_lgb_cli, 5)
submission["is_installed"] = np.round(predictions_lgb_ins, 5)
submission.to_csv('./output/lgb_42_sep_train_logloss_float_round5.csv', index=False, sep='\t')

In [28]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = np.random.random((test_data.shape[0]))
submission["is_installed"] = predictions_lgb_ins
submission.to_csv('./output/lgb_42_sep_train_logloss_float_ins_pred_cli_rand.csv', index=False, sep='\t')

In [27]:
submission

,RowId,is_clicked,is_installed
0,64505,0.54069,0.35231
1,64506,0.20442,0.45921
2,64507,0.63765,0.08273
3,64508,0.30246,0.33364
4,64509,0.91326,0.30514
...,...,...,...
160968,16240,0.67619,0.42696
160969,16241,0.09272,0.00923
160970,16242,0.22014,0.02216
160971,16243,0.50215,0.14860
